# Project: Profitable App Profiles for the App Store and Google Play Markets
## Introduction
My aim in this project is to find mobile app profiles that are profitable for the App Store and Google Play markets. I am working as data analysts for a company that builds Android and iOS mobile apps, and my job is to enable a team of developers to make data-driven decisions with respect to the kind of apps they build.

At my company, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. This means that our revenue for any given app is mostly influenced by the number of users that use our app. My goal for this project is to analyze data to help my company's developers understand what kinds of apps are likely to attract more users.



## The Data
As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play.

Collecting data for over four million apps requires a significant amount of time and money, so I'll try to analyze a sample of data instead. 

To avoid spending resources with collecting new data myself, I should first try to see whether I can find any relevant existing data at no cost. Luckily, these are two data sets that seem suitable for my purpose:

A [data set](https://www.kaggle.com/lava18/google-play-store-apps/home) containing data about approximately ten thousand Android apps from Google Play
A [data set](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home) containing data about approximately seven thousand iOS apps from the App Store

Using this data, I'll create a function I can use to examine the data throughout this project. This function takes the dataset name in as an argument as well as a start and end row index. These three parameters are used to show a preview of the data sets I'll be working with. Also, the 'rows_and_columns' parameter will give me the dimensions of the data set when set to True.

In [42]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

Here I'll open the Apple App Store data set and use the 'explore_data' function to preview the data

In [36]:
opened_file = open('AppleStore.csv')
from csv import reader
read_file = reader(opened_file)

apple_store_apps = list(read_file)

explore_data(app_store_apps, 0, 3)

apple_headers = app_store_apps[0]
print(apple_headers)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


Number of rows: 7198
Number of columns: 16
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


I'll do the same with the Google Play Store data set

In [13]:
opened_file_2 = open('googleplaystore.csv')
read_file_2 = reader(opened_file_2)

play_store_apps = list(read_file_2)
explore_data(play_store_apps, 0, 3)

google_headers = play_store_apps[0]

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


Number of rows: 10842
Number of columns: 13


## Methods
Prior to performing any analyses, the data sets will need to be cleaned and filtered so any unnecessary data will be removed and will not affect my results.

### Cleaning the data
From reading the discussion forum for the Google Play Store data set, I know the row with the index 10473 is missing data. I'll use the 'del' function to delete this row from the data set.

In [15]:
#deleting a row with missing information using the 'del' function
print(play_store_apps[10473])
#del play_store_apps[10473]

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


### Removing Duplicates
The Google Play data set has duplicate entries for some apps. Having duplicate entries for the same app could influence my analysis later, so I will be checking for duplicates using the for loop below.

In [17]:
duplicate_apps = []
unique_apps = []

for app in play_store_apps[1:]:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
        
print('Number of duplicate apps:', len(duplicate_apps))
print('\n')
print('Examples of duplicate apps:', duplicate_apps[:10])

Number of duplicate apps: 1181


Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack']


Using the the above block of code, we can see there are 1181 apps in the Google Play Store that have duplicates. To remove these duplicates, I will be examining the number of reviews for each duplicate app and keep the entries with the highest number of reviews. I am assuming that more reviews indicate more recent data.

In [21]:
reviews_max = {}

for app in play_store_apps[1:]:
    name = app[0]
    n_reviews = float(app[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews


9659


Now I will check the length of reviews_max to make sure the dictionary doesn't contain the duplicates

In [23]:
print('Expected length:', len(play_store_apps[1:]) - 1181)
print('Actual length:', len(reviews_max))

Expected length: 9659
Actual length: 9659


Now I will use the dictionary I created to remove the duplicate rows. 

I do this by creating two empty lists: play_store_clean and already_added. I loop through the Google Play dataset, and if the number of reviews of an app is the same as the maximum reviews of an app in the 'reviews_max' dictionary I created above, and if the app name is not already in the list 'already_added', the app row gets appended to the 'play_store_clean' list.

This part of the loop checks the whole Google Play Store data set and only adds entries that match the 'max_reviews' dictionary entries to the 'play_store_clean' list.

The part of the loop that appends the 'already_added' list, adds the app name to the list if it has already been added to the 'play_store_clean' list.

In [26]:
play_store_clean = []
already_added = []

for app in play_store_apps[1:]:
    name = app[0]
    n_reviews = float(app[3])
    
    if (n_reviews == reviews_max[name]) and (name not in already_added):
        play_store_clean.append(app)
        already_added.append(name)

I use the 'explore_data' function to check the 'play_store_clean' list to make sure it is the expected length.

In [45]:
explore_data(play_store_clean, 0, 3)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']




### Filtering Data: Apps in English
For the purposes of this analysis, I am only looking at apps directed at English-speaking audiences. So I must filter out apps with names that suggest they are not directed toward English-speakers.
To do this, I created the function 'english_check' which takes in a string and checks if the string has more than three characters that fall outside the ASCII range (0 - 127). If the  function should returns False it identifies the string as non-English because it has more than three characters outside the English ASCII range, otherwise it should return True.

In [50]:
def english_check(string):
    non_ascii = 0
    
    for character in string:
        if ord(character) > 127:
            non_ascii += 1
    
    if non_ascii > 3:
        return False
    else:
        return True

Here I will loop through both data sets and use the function I created above to create two new lists that have non-English apps filtered out.

In [52]:
play_store_english = []
apple_store_english = []

for app in play_store_clean:
    name = app[0]
    if english_check(name):
        play_store_english.append(app)
        
for app in apple_store_apps:
    name = app[1]
    if english_check(name):
        apple_store_english.append(app)
        
explore_data(play_store_english, 0, 3, True)
explore_data(apple_store_english, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9614
Number of columns: 13
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram

### Filtering Data: Free Apps

Now that inaccurate, duplicate, and non-English entries have been removed from both data sets, I still need to isolate free apps in their own lists. I will do this using the for loop below.

I create two empty lists, which will be populated with the apps that meet the criteria in the for loops.

In [54]:
free_play_store_apps = []
free_apple_store_apps = []

for app in play_store_english:
    price = app[7]
    
    if price == '0':
        free_play_store_apps.append(app)
        
for app in apple_store_english:
    price = app[4]
    
    if price == '0.0':
        free_apple_store_apps.append(app)
        
print('Number of free Google Play apps:', len(free_play_store_apps))
print('Number of free Apple Store apps:', len(free_apple_store_apps))

Number of free Google Play apps: 8864
Number of free Apple Store apps: 3222


## Analysis

As mentioned in the introduction, my aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we develop it further.
3. If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

### Most Common Apps by Genre

Because our end goal is to add the app on both Google Play and the App Store, I need to find app profiles that are successful on both markets. To do this, I need to get a sense of what the most common genres are for each market. I'll create frequency tables for a few columns in the data sets to do this.

First I'll examine the column headers to identify which columns I could use to generate frequency tables.

In [56]:
print("Apple App Store Columns:", apple_headers)
print('\n')
print("Google Play Store Columns", google_headers)

Apple App Store Columns: ['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


Google Play Store Columns ['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


Looking at the column headers, 'prime_genre' for the Apple Store, and 'Category' and 'Genre' for the Google Play Store seem like good candidates for generating a frequency table.

Below, I created two functions. One generates a frequency table by populating an empty dictionary 'table' with value counts of unique values within a specified column. The function then caluclates the frequencies as percentages  and returns the frequency table as a dictionary.

The second function generates the frequency table of percentages in descending order using the freq_table function.

In [59]:
def freq_table(dataset, index):
    table = {}
    total = 0
    
    for row in dataset:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
    
    table_percentages = {}
    for key in table:
        percentage = (table[key] / total) * 100
        table_percentages[key] = percentage 
    
    return table_percentages

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

I'll use the two functions to display the frequency table of 'prime_genre', 'Genres', and 'Category' columns.

### Analyzing Apple App Store Frequencies
Based on the frequency table below, it seems that Games apps are the most popular in the Apple App Store. So much so that Games make up 58% of apps in the app store alone. Entertainment apps are the second most common app, making up almost 8% of the app store apps.

It also seems that entertainment and media are more common than utility apps such as productivity, news, finance, weather, and reference apps. Although, the Games genre makes up more than half the apps in the Apple App Store, it is important to remember that this table represents frequency within the store and not popularity among users.

In [60]:
#index 11 is 'Prime_Genre'
display_table(free_apple_store_apps, 11)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


### Analyzing Google Play Store Frequencies
When looking at the table for Category, we see there are several more categories than in the Apple App Store. When looking at the Genres table we see that categories are broken down even further. 'Tools' shows up in the top three most common categories for both columns. 

The 'Family' category is the most common (18%) in the Google Play Store and 'Tools' is the most common genre (8%). There is not one category or genre that makes up over half the market like in the Apple App Store. Also, more practical apps (Tools, Business, Lifestyle, Productivity) seem to be more common than entertainment-like apps. Similarly, we cannot assume that these practical apps are more popular or have more users, they are simply the most common app types within the Google Play Store.

In [61]:
#index 1 is 'Category'
display_table(free_play_store_apps, 1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

In [62]:
#index 9 = 'Genre'
display_table(free_play_store_apps, 9)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

Based on the frequency tables generated, I cannot recommoned an app profile yet. I still need more information, including apps that have the most users.

### Apps with the Most Users
The information presented in the frequency tables showed which app genres were most common, but not most popular. To find which apps have the most users, I'll need to calculate the average number of installs for each app genre. For the Google Play data set, I can find this information in the 'Installs' column, but this information is missing for the App Store data set. As a workaround, I'll take the total number of user ratings as a proxy, which I can find in the 'rating_count_tot' column.

### Apple App Store

To calculate the average number of user ratings for each genre, I'll use a nested for loop.

In [63]:
apple_store_genres = freq_table(free_apple_store_apps, -5)

for genre in apple_store_genres:
    total = 0
    len_genre = 0
    for app in free_apple_store_apps:
        app_genre = app[-5]
        if app_genre == genre:            
            n_ratings = float(app[5])
            total += n_ratings
            len_genre += 1
    avg_n_ratings = total / len_genre
    print(genre, ':', avg_n_ratings)

Photo & Video : 28441.54375
Travel : 28243.8
Utilities : 18684.456790123455
Reference : 74942.11111111111
Medical : 612.0
Navigation : 86090.33333333333
Weather : 52279.892857142855
Catalogs : 4004.0
Music : 57326.530303030304
Sports : 23008.898550724636
Lifestyle : 16485.764705882353
Productivity : 21028.410714285714
Health & Fitness : 23298.015384615384
Education : 7003.983050847458
Business : 7491.117647058823
Book : 39758.5
Food & Drink : 33333.92307692308
Games : 22788.6696905016
Social Networking : 71548.34905660378
Entertainment : 14029.830708661417
News : 21248.023255813954
Finance : 31467.944444444445
Shopping : 26919.690476190477


Based on these results, it looks like Navigation has the highest average of user ratings along with with Social Networking, Music, and Weather. Considering giant companies like Waze, Facebook, and Spotify, a Weather app may be a good way to go.

In [66]:
for app in free_apple_store_apps:
    if app[-5] == 'Weather':
        print(app[1], ':', app[5])

The Weather Channel: Forecast, Radar & Alerts : 495626
The Weather Channel App for iPad – best local forecast, radar map, and storm tracking : 208648
WeatherBug - Local Weather, Radar, Maps, Alerts : 188583
MyRadar NOAA Weather Radar Forecast : 150158
AccuWeather - Weather for Life : 144214
Yahoo Weather : 112603
Weather Underground: Custom Forecast & Local Radar : 49192
NOAA Weather Radar - Weather Forecast & HD Radar : 45696
Weather Live Free - Weather Forecast & Alerts : 35702
Storm Radar : 22792
QuakeFeed Earthquake Map, Alerts, and News : 6081
Moji Weather - Free Weather Forecast : 2333
Hurricane by American Red Cross : 1158
Forecast Bar : 375
Hurricane Tracker WESH 2 Orlando, Central Florida : 203
FEMA : 128
iWeather - World weather forecast : 80
Weather - Radar - Storm with Morecast App : 78
Yurekuru Call : 53
Weather & Radar : 37
WRAL Weather Alert : 25
Météo-France : 24
JaxReady : 22
Freddy the Frogcaster's Weather Station : 14
Almanac Long-Range Weather Forecast : 12
TodayAir

It seems like the Weather category is also dominated by big companies like The Weather Channel and agencies such as NOAA. Other categories are also dominated by established companies. Creating a game would be a good idea if the app market wasn't so saturated with them already. Combining Reference and Books could be a way to go.

In [67]:
for app in free_apple_store_apps:
    if app[-5] == 'Reference':
        print(app[1], ':', app[5])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


In [69]:
for app in free_apple_store_apps:
    if app[-5] == 'Book':
        print(app[1], ':', app[5])

Kindle – Read eBooks, Magazines & Textbooks : 252076
Audible – audio books, original series & podcasts : 105274
Color Therapy Adult Coloring Book for Adults : 84062
OverDrive – Library eBooks and Audiobooks : 65450
HOOKED - Chat Stories : 47829
BookShout: Read eBooks & Track Your Reading Goals : 879
Dr. Seuss Treasury — 50 best kids books : 451
Green Riding Hood : 392
Weirdwood Manor : 197
MangaZERO - comic reader : 9
ikouhoushi : 0
MangaTiara - love comic reader : 0
謎解き : 0
謎解き2016 : 0


It looks like the Bible and Dictionary are the most popular in Reference, and software that accesses books is the most popular in Books. Combining these two seems like a good niche market to enter.

### Google Play Store
Now I'll look at the data from the Google Play Store.

In [71]:
categories_google = freq_table(free_play_store_apps, 1)

for category in categories_google:
    total = 0
    len_category = 0
    for app in free_play_store_apps:
        category_app = app[1]
        if category_app == category:            
            n_installs = app[5]
            n_installs = n_installs.replace(',', '')
            n_installs = n_installs.replace('+', '')
            total += float(n_installs)
            len_category += 1
    avg_n_installs = total / len_category
    print(category, ':', avg_n_installs)

EDUCATION : 1833495.145631068
NEWS_AND_MAGAZINES : 9549178.467741935
LIBRARIES_AND_DEMO : 638503.734939759
DATING : 854028.8303030303
HEALTH_AND_FITNESS : 4188821.9853479853
COMMUNICATION : 38456119.167247385
LIFESTYLE : 1437816.2687861272
FAMILY : 3695641.8198090694
SHOPPING : 7036877.311557789
HOUSE_AND_HOME : 1331540.5616438356
COMICS : 817657.2727272727
FOOD_AND_DRINK : 1924897.7363636363
PARENTING : 542603.6206896552
EVENTS : 253542.22222222222
MAPS_AND_NAVIGATION : 4056941.7741935486
BEAUTY : 513151.88679245283
GAME : 15588015.603248259
BOOKS_AND_REFERENCE : 8767811.894736841
MEDICAL : 120550.61980830671
ENTERTAINMENT : 11640705.88235294
ART_AND_DESIGN : 1986335.0877192982
SOCIAL : 23253652.127118643
BUSINESS : 1712290.1474201474
SPORTS : 3638640.1428571427
VIDEO_PLAYERS : 24727872.452830188
FINANCE : 1387692.475609756
TOOLS : 10801391.298666667
WEATHER : 5074486.197183099
PHOTOGRAPHY : 17840110.40229885
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
AUTO_

We can see the same patterns that were in the Apple App Store: Social, 

In [72]:
for app in free_play_store_apps:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

Again, this market seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there'll be some significant competition.

We also notice there are quite a few apps built around the book Quran, which suggests that building an app around a popular book can be profitable. It seems that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets.

However, it looks like the market is already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.

## Conclusion
In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc